Import common libraries

In [1]:
import mlflow

Models with column based inputs work as is (checkout example below - sklearn). Look at the following examples for models with tensor based inputs work when logged as pyfunc (checkout examples below - xgboost, pytorch, tensorflow and simple python functions). 

Use the test data (example, X_test for xgboost) or the first element of the test data for sending a request to these models. Once you register them and create a model API in Domino, if you are using the tester on the UI, make sure they are properly formatted and the request is of the form : {"data": your test data}

Logging models using the following templates works with our model APIs. 

XGBOOST (array input)

In [4]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

data = load_iris()
X_train, X_test, y_train, y_test = train_test_split(
    data["data"], data["target"], test_size=0.2
)

xgb_classifier = XGBClassifier(
    n_estimators=10,
    max_depth=3,
    learning_rate=1,
    objective="binary:logistic",
    random_state=123,
)

# train model
xgb_classifier.fit(X_train, y_train)

class SomeModel(mlflow.pyfunc.PythonModel):
    def __init__(self,model):
        self.model = model
    # Define a function that returns prediction
    def nn_func(x):
        return self.model.predict(x)

model = SomeModel(xgb_classifier)

# Create the Pyfunc and log it to MLflow
with mlflow.start_run() as run:
    model_info = mlflow.pyfunc.log_model(
        registered_model_name="pyfunc-xgboost-model", 
        python_model=model,
        artifact_path="test-model"
    )
print(model_info)

Registered model 'pyfunc-xgboost-model' already exists. Creating a new version of this model...
2023/09/11 18:37:55 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: pyfunc-xgboost-model, version 2
Created version '2' of model 'pyfunc-xgboost-model'.


In [10]:
signature

inputs: 
  [Tensor('float64', (-1, 4))]
outputs: 
  [Tensor('int64', (-1,))]
params: 
  None

SKLEARN (column based inputs)

In [3]:
from sklearn.linear_model import LinearRegression
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split


# loading the California housing dataset
cali_housing = fetch_california_housing(as_frame=True)

# split the dataset into train and test partitions
X_train, X_test, y_train, y_test = train_test_split(
    cali_housing.data, cali_housing.target, test_size=0.2, random_state=123
)

# train the model
lin_reg = LinearRegression().fit(X_train, y_train)

# Infer model signature
predictions = lin_reg.predict(X_train)
signature = infer_signature(X_train, predictions)


with mlflow.start_run() as run:
    mlflow.sklearn.log_model(lin_reg, registered_model_name="sklearn-model", artifact_path="sklearn-model", signature=signature)
mlflow.end_run()

Successfully registered model 'sklearn-model'.
2023/08/31 22:27:39 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: sklearn-model, version 1
Created version '1' of model 'sklearn-model'.


In [4]:
signature

inputs: 
  ['MedInc': double, 'HouseAge': double, 'AveRooms': double, 'AveBedrms': double, 'Population': double, 'AveOccup': double, 'Latitude': double, 'Longitude': double]
outputs: 
  [Tensor('float64', (-1,))]
params: 
  None

TENSORFLOW (tensor based inputs)

In [11]:
import tensorflow as tf

mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential(
    [
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10),
    ]
)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer="adam", loss=loss_fn, metrics=["accuracy"])

# train model 
model.fit(x_train, y_train, epochs=5)

class SomeModel(mlflow.pyfunc.PythonModel):
    def __init__(self,model):
        self.model = model
    # Define a function that returns prediction
    def nn_func(x):
        return self.model.predict(x)

model_tf = SomeModel(model)

# Create the Pyfunc and log it to MLflow
with mlflow.start_run() as run:
    model_info = mlflow.pyfunc.log_model(
        registered_model_name="pyfunc-tensorflow-model", 
        python_model=model_tf,
        artifact_path="test-model",
        pip_requirements=["tensorflow"]
    )
print(model_info)

2023-09-07 23:53:18.487915: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-07 23:53:19.745071: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


11490434/11490434 [==============================] - 1s 0us/step
Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2939 - accuracy: 0.9151
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1426 - accuracy: 0.9582
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1059 - accuracy: 0.9679
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0874 - accuracy: 0.9732
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0742 - accuracy: 0.9771


/opt/conda/lib/python3.9/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'pyfunc-tensorflow-model'.
2023/09/07 23:53:50 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: pyfunc-tensorflow-model, version 1
Created version '1' of model 'pyfunc-tensorflow-model'.


In [24]:
signature

inputs: 
  [Tensor('float64', (-1, 28, 28))]
outputs: 
  [Tensor('float32', (-1, 10))]
params: 
  None

PYTORCH (tensor based inputs)

In [3]:
import mlflow.pyfunc
import numpy as np
import torch
from torch import nn

net = nn.Linear(6, 1)
loss_function = nn.L1Loss()
optimizer = torch.optim.Adam(net.parameters(), lr=1e-4)

X = torch.randn(6)
y = torch.randn(1)

# train model 
epochs = 5
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = net(X)

    loss = loss_function(outputs, y)
    loss.backward()

    optimizer.step()

class SomeModel(mlflow.pyfunc.PythonModel):
    def __init__(self,model):
        self.model = model
    # Define a function that returns prediction
    def nn_func(x):
        return self.model(x).detach().numpy()

model = SomeModel(net)

# Create the Pyfunc and log it to MLflow
with mlflow.start_run() as run:
    model_info = mlflow.pyfunc.log_model(
        registered_model_name="pyfunc-torch-model-4", # important,
        python_model=model,
        artifact_path="test-model-4",
        pip_requirements=["torch"]
    )
print(model_info)

Successfully registered model 'pyfunc-torch-model-4'.
2023/09/07 21:56:24 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: pyfunc-torch-model-4, version 1
Created version '1' of model 'pyfunc-torch-model-4'.


In [13]:
signature

inputs: 
  [Tensor('float32', (-1,))]
outputs: 
  [Tensor('float32', (-1,))]
params: 
  None

CUSTOM PYTHON FUNCTION (tensor based inputs)

Simple python function with artifact

In [ ]:
import mlflow.pyfunc
import numpy as np
import pandas as pd

class SomeModel(mlflow.pyfunc.PythonModel):
    # Define a function that takes in a NumPy array and returns its sum
    def sum_func(arr):
        return np.sum(arr)

# Define a function that generates an artifact in memory
def generate_artifact():
    data = pd.DataFrame({"a": [1, 2, 3], "b": [4, 5, 6]})
    return data.to_csv("data.csv", index=False)

generate_artifact()
model = SomeModel()

# Create the Pyfunc and log it to MLflow
with mlflow.start_run() as run:
    model_info = mlflow.pyfunc.log_model(
        registered_model_name="test", 
        python_model=model,
        artifact_path="test-model",
        artifacts={"model_file": "data.csv"}
    )
print(model_info)

Simple python function without artifact 

In [ ]:
class SomeModel(mlflow.pyfunc.PythonModel):
    # Define a function that takes in a NumPy array and returns its sum
    def sum_func(arr):
        return np.sum(arr)

# # Define a function that generates an artifact in memory
# def generate_artifact():
#     data = pd.DataFrame({"a": [1, 2, 3], "b": [4, 5, 6]})
#     return data.to_csv("data.csv", index=False)

# generate_artifact()
model = SomeModel()

# Create the Pyfunc and log it to MLflow
with mlflow.start_run() as run:
    model_info = mlflow.pyfunc.log_model(
        registered_model_name="test-wo-artifact",
        python_model=model,
        artifact_path="test-model-1"
        #artifacts={"model_file": "data.csv"}
    )
print(model_info)